In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import h5py
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
import xarray as xr
import matplotlib.pyplot as plt
import statistics as st
import random
import warnings
#import pydot

sim_dir = '/qdata1/pbrown/dbaseV7/simV7x_tms/1810'
#month = '1811'

#print('Files Available:')
#for ifile in os.listdir(sim_dir):
#for ifile in os.listdir(sim_dir+'/'+month):
#  print(ifile)
    
sim_fname = 'TMS.dbsatTb.20181001.026079.sim'
sim_file = sim_dir+'/'+sim_fname
#sim_file = sim_dir+month+'/'+sim_fname


MHSfreqs = np.zeros(5,dtype='f')
MHSviewangles   = np.zeros(10,dtype='f')

with open(sim_file,'rb') as sfile:
    
    sat_id = sfile.read(5)
    sensor = sfile.read(5)
    
    MHSfreqs         = np.fromfile(sfile, dtype='f', count=5)
    MHSviewangles    = np.fromfile(sfile, dtype='f', count=10)

    pix_start        = np.fromfile(sfile, dtype='i', count=1)
    pix_end          = np.fromfile(sfile, dtype='i', count=1)
    scan_start       = np.fromfile(sfile, dtype='i', count=1)
    scan_end         = np.fromfile(sfile, dtype='i', count=1)

    npix   = int(pix_end - pix_start)
    nscans = int(scan_end - scan_start)

    print('File Header Info:')
    print('---------------------------------------------------')
    print(sat_id,sensor)
    print('MHS Frequencies: ')
    print(MHSfreqs)
    print('MHS View Angles: ')
    print(MHSviewangles)
    print('')
    print('Pix Start    = ', pix_start)
    print('Pix End      = ', pix_end)
    print('Scan Start   = ', scan_start)
    print('Scan End     = ', scan_end)
    print('----------------------------------------------------')

    pix         = np.zeros(1, dtype='i')
    scn         = np.zeros(1, dtype='i')
    lats        = np.zeros([npix,nscans],dtype='f')
    lons        = np.zeros([npix,nscans],dtype='f')
    elev        = np.zeros([npix,nscans],dtype='f')
    scan_time   = np.zeros([nscans,6],dtype='i')
    sfc_type    = np.zeros([npix,nscans],dtype='i')
    chi_sqr     = np.zeros([npix,nscans], dtype='f')
    mirs_qc     = np.zeros([npix,nscans], dtype='i')
    ds_fpa      = np.zeros([npix,nscans,10],dtype='f')
    msfc_precip = np.zeros([npix,nscans,10],dtype='f')
    csfc_precip = np.zeros([npix,nscans,10],dtype='f')
    bsfc_precip = np.zeros([npix,nscans,10],dtype='f')
    conv_precip = np.zeros([npix,nscans,10],dtype='f')
    emiss_fpa   = np.zeros([5,npix,nscans,10],dtype='f')
    rwc28       = np.zeros([28,npix,nscans],dtype='f')
    swc28       = np.zeros([28,npix,nscans],dtype='f')
    cwc28       = np.zeros([28,npix,nscans],dtype='f')
    SLH28       = np.zeros([28,npix,nscans],dtype='f')
    dTbssim_fpa = np.zeros([5,npix,nscans,10],dtype='f')
    biasTbs     = np.zeros([5,npix,nscans],dtype='f')
    nchans      = 5
    nangs       = 10
    
#    dTbssim_fpa = np.nan
    
    outer = 0

    print('Beginning read loop...')

    

    for iscan in np.arange(0,nscans):
        for ipix in np.arange(0,npix):
            pix = np.fromfile(sfile, dtype='i', count=1)
            scn = np.fromfile(sfile, dtype='i', count=1)
            #print('pix : ', pix)
            
            #print('scan : ', scn)


            if np.size(pix) == 0 :
                last_pix = ipix - 1
                last_scan = iscan
                print('File end at scan, pixel = '+str(last_scan)+','+str(last_pix))
                outer = 1
                break

            
            lats[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('lat : ', lats[ipix,iscan])
            lons[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('lon : ', lons[ipix,iscan])
            elev[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('elev : ', elev[ipix,iscan])
            scan_time[iscan,0:6]      = np.fromfile(sfile, dtype='i', count=6)
            #print('scan time : ', scan_time[ipix,...])
            sfc_type[ipix,iscan]      = np.fromfile(sfile, dtype='i', count=1)
            #print('surface type : ', sfc_type[ipix,iscan])
            #print(sfc_type[ipix,iscan])
#            chi_sqr[ipix,iscan]       = np.fromfile(sfile, dtype='f', count=1)
#            print('chi_squared : ', chi_sqr[ipix,iscan])
#            mirs_qc[ipix,iscan]       = np.fromfile(sfile, dtype='i', count=1)
#            print('MIRS_qc : ', mirs_qc[ipix,iscan])
            ds_fpa[ipix,iscan,0:10]        = np.fromfile(sfile, dtype='f', count=10)
            #print('ds_fpa : ', ds_fpa[ipix,iscan,...])
            #print(ds_fpa[ipix,iscan,...])
            ##sfc_precip[ipix,iscan]    = np.fromfile(sfile, dtype='f', count=1)
            msfc_precip[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('msfc_precip : ', msfc_precip[ipix,iscan,...])
            csfc_precip[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('csfc_precip : ', csfc_precip[ipix,iscan,...])
            bsfc_precip[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('bsfc_precip : ', bsfc_precip[ipix,iscan,...])
            conv_precip[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('conv_precip : ', conv_precip[ipix,iscan,...])
            ##conv_frac[ipix,iscan]     = np.fromfile(sfile, dtype='f', count=1)
            ##fill                      = np.fromfile(sfile, dtype='f', count=1)
            #ichan = 0
            for iang in np.arange(0,nangs):
                emiss_fpa[0:5,ipix,iscan,iang] = np.fromfile(sfile, dtype='f', count=5)
                #print(emiss_fpa[ichan,ipix,iscan,...])
                if iang == 10:
                    break
            #print(emiss_fpa[0,ipix,iscan,...])
            #print(emiss_fpa[1,ipix,iscan,...])
            #print(emiss_fpa[2,ipix,iscan,...])
            #print(emiss_fpa[3,ipix,iscan,...])
            #print(emiss_fpa[4,ipix,iscan,...])
            rwc28[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('rwc : ', rwc28[...,ipix,iscan])
            swc28[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('swc : ', swc28[...,ipix,iscan])
            cwc28[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('cwc : ', cwc28[...,ipix,iscan])
            SLH28[0:28,ipix,iscan]  = np.fromfile(sfile, dtype='f', count=28)
#            for jchan in np.arange(0,nchans):  
#                dTbssim_fpa[jchan,ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
#                #print('dTbssim_fpa : ', dTbssim_fpa[jchan,ipix,iscan,...])
#                if jchan == 5:
#                    break
            for jang in np.arange(0,nangs):  
                dTbssim_fpa[0:5,ipix,iscan,jang] = np.fromfile(sfile, dtype='f', count=5)
                #print('angle : ', jang)
                #print('dTbssim_fpa : ', dTbssim_fpa[0:5,ipix,iscan,jang])
                if jang == 10:
                    break
            biasTbs[0:5,ipix,iscan] = np.fromfile(sfile, dtype='f', count=5)
            #print('bias Tbs : ', biasTbs[...,ipix,iscan])
            
#            STOP

            
        if outer == 1:
            break

print('Finished Read Loop')

print('Did it work?')

In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import h5py
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import datetime
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.inspection import permutation_importance
import xarray as xr
import matplotlib.pyplot as plt
import statistics as st
import random
import warnings
#import pydot

sim_dir = '/qdata1/pbrown/dbaseV7/simV7x_atms/1810'
#month = '1811'

#print('Files Available:')
#for ifile in os.listdir(sim_dir):
#for ifile in os.listdir(sim_dir+'/'+month):
#  print(ifile)
    
sim_fname = 'ATMS.dbsatTb.20181001.026079.sim'
sim_file = sim_dir+'/'+sim_fname
#sim_file = sim_dir+month+'/'+sim_fname


MHSfreqsa = np.zeros(5,dtype='f')
MHSviewanglesa   = np.zeros(10,dtype='f')

with open(sim_file,'rb') as sfile:
    
    sat_ida = sfile.read(5)
    sensora = sfile.read(5)
    
    MHSfreqsa         = np.fromfile(sfile, dtype='f', count=5)
    MHSviewanglesa    = np.fromfile(sfile, dtype='f', count=10)

    pix_starta        = np.fromfile(sfile, dtype='i', count=1)
    pix_enda          = np.fromfile(sfile, dtype='i', count=1)
    scan_starta       = np.fromfile(sfile, dtype='i', count=1)
    scan_enda         = np.fromfile(sfile, dtype='i', count=1)

    npixa   = int(pix_end - pix_start)
    nscansa = int(scan_end - scan_start)

    print('File Header Info:')
    print('---------------------------------------------------')
    print(sat_ida,sensora)
    print('MHS Frequencies: ')
    print(MHSfreqsa)
    print('MHS View Angles: ')
    print(MHSviewanglesa)
    print('')
    print('Pix Start    = ', pix_starta)
    print('Pix End      = ', pix_enda)
    print('Scan Start   = ', scan_starta)
    print('Scan End     = ', scan_enda)
    print('----------------------------------------------------')

    pixa         = np.zeros(1, dtype='i')
    scna         = np.zeros(1, dtype='i')
    latsa        = np.zeros([npix,nscans],dtype='f')
    lonsa        = np.zeros([npix,nscans],dtype='f')
    eleva        = np.zeros([npix,nscans],dtype='f')
    scan_timea   = np.zeros([nscans,6],dtype='i')
    sfc_typea    = np.zeros([npix,nscans],dtype='i')
    chi_sqra     = np.zeros([npix,nscans], dtype='f')
    mirs_qca     = np.zeros([npix,nscans], dtype='i')
    ds_fpaa      = np.zeros([npix,nscans,10],dtype='f')
    msfc_precipa = np.zeros([npix,nscans,10],dtype='f')
    csfc_precipa = np.zeros([npix,nscans,10],dtype='f')
    bsfc_precipa = np.zeros([npix,nscans,10],dtype='f')
    conv_precipa = np.zeros([npix,nscans,10],dtype='f')
    emiss_fpaa   = np.zeros([5,npix,nscans,10],dtype='f')
    rwc28a       = np.zeros([28,npix,nscans],dtype='f')
    swc28a       = np.zeros([28,npix,nscans],dtype='f')
    cwc28a       = np.zeros([28,npix,nscans],dtype='f')
    SLH28a       = np.zeros([28,npix,nscans],dtype='f')
    dTbssim_fpaa = np.zeros([5,npix,nscans,10],dtype='f')
    biasTbsa     = np.zeros([5,npix,nscans],dtype='f')
    nchansa       = 5
    nangsa       = 10
    
    
    outera = 0

    print('Beginning read loop...')

    

    for iscan in np.arange(0,nscansa):
        for ipix in np.arange(0,npixa):
            pixa = np.fromfile(sfile, dtype='i', count=1)
            scna = np.fromfile(sfile, dtype='i', count=1)
            #print('pix : ', pix)
            
            #print('scan : ', scn)


            if np.size(pixa) == 0 :
                last_pixa = ipix - 1
                last_scana = iscan
                print('File end at scan, pixel = '+str(last_scana)+','+str(last_pixa))
                outera = 1
                break

            
            latsa[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('lat : ', latsa[ipix,iscan])
            lonsa[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('lon : ', lonsa[ipix,iscan])
            eleva[ipix,iscan]          = np.fromfile(sfile, dtype='f', count=1)
            #print('elev : ', eleva[ipix,iscan])
            scan_timea[iscan,0:6]      = np.fromfile(sfile, dtype='i', count=6)
            #print('scan time : ', scan_timea[ipix,...])
            sfc_typea[ipix,iscan]      = np.fromfile(sfile, dtype='i', count=1)
            #print('surface type : ', sfc_typea[ipix,iscan])
            #print(sfc_type[ipix,iscan])
#            chi_sqr[ipix,iscan]       = np.fromfile(sfile, dtype='f', count=1)
#            print('chi_squared : ', chi_sqr[ipix,iscan])
#            mirs_qc[ipix,iscan]       = np.fromfile(sfile, dtype='i', count=1)
#            print('MIRS_qc : ', mirs_qc[ipix,iscan])
            ds_fpaa[ipix,iscan,0:10]        = np.fromfile(sfile, dtype='f', count=10)
            #print('ds_fpa : ', ds_fpaa[ipix,iscan,...])
            #print(ds_fpa[ipix,iscan,...])
            ##sfc_precip[ipix,iscan]    = np.fromfile(sfile, dtype='f', count=1)
            msfc_precipa[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('msfc_precip : ', msfc_precipa[ipix,iscan,...])
            csfc_precipa[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('csfc_precip : ', csfc_precipa[ipix,iscan,...])
            bsfc_precipa[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('bsfc_precip : ', bsfc_precipa[ipix,iscan,...])
            conv_precipa[ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
            #print('conv_precip : ', conv_precipa[ipix,iscan,...])
            ##conv_frac[ipix,iscan]     = np.fromfile(sfile, dtype='f', count=1)
            ##fill                      = np.fromfile(sfile, dtype='f', count=1)
            #ichan = 0
            for iang in np.arange(0,nangsa):
                emiss_fpaa[0:5,ipix,iscan,iang] = np.fromfile(sfile, dtype='f', count=5)
                #print(emiss_fpaa[ichan,ipix,iscan,...])
                if iang == 10:
                    break
            #print(emiss_fpa[0,ipix,iscan,...])
            #print(emiss_fpa[1,ipix,iscan,...])
            #print(emiss_fpa[2,ipix,iscan,...])
            #print(emiss_fpa[3,ipix,iscan,...])
            #print(emiss_fpa[4,ipix,iscan,...])
            rwc28a[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('rwc : ', rwc28a[...,ipix,iscan])
            swc28a[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('swc : ', swc28a[...,ipix,iscan])
            cwc28a[0:28,ipix,iscan]         = np.fromfile(sfile, dtype='f', count=28)
            #print('cwc : ', cwc28a[...,ipix,iscan])
            SLH28a[0:28,ipix,iscan]  = np.fromfile(sfile, dtype='f', count=28)
#            for jchan in np.arange(0,nchans):  
#                dTbssim_fpaa[jchan,ipix,iscan,0:10] = np.fromfile(sfile, dtype='f', count=10)
#                #print('dTbssim_fpa : ', dTbssim_fpa[jchan,ipix,iscan,...])
#                if jchan == 5:
#                    break
            for jang in np.arange(0,nangs):  
                dTbssim_fpaa[0:5,ipix,iscan,jang] = np.fromfile(sfile, dtype='f', count=5)
                #print('angle : ', jang)
                #print('dTbssim_fpa : ', dTbssim_fpaa[0:5,ipix,iscan,jang])
                if jang == 10:
                    break
            biasTbsa[0:5,ipix,iscan] = np.fromfile(sfile, dtype='f', count=5)
            #print('bias Tbs : ', biasTbsa[...,ipix,iscan])
            
#            STOP

            
        if outera == 1:
            break

print('Finished Read Loop')

print('Did it work?')

In [ ]:
x  = np.zeros([8000,20,10,5],dtype='f') #[scan,pix,scanang,chan]
y  = np.zeros([8000,20,10,5],dtype='f')

x[:] = np.nan
y[:] = np.nan

for ichan in np.arange(0,chan):
    for iscan in np.arange(0,last_scan):
        for ipix in np.arange(0,npix):
            for iang in np.arange(0,scnang):
                x[iscan,ipix,iang,ichan] = dTbssim_fpa[ichan,ipix,iscan,iang]

for ichan in np.arange(0,chan):
    for iscan in np.arange(0,last_scana):
        for ipix in np.arange(0,npix):
            for iang in np.arange(0,scnang):
                y[iscan,ipix,iang,ichan] = dTbssim_fpaa[ichan,ipix,iscan,iang]
                
mask_x = np.isfinite(x).all(axis=(2,3))
mask_y = np.isfinite(y).all(axis=(2,3))

mask = mask_x*mask_y

x_valid = x[mask]
y_valid = y[mask]

ds = xr.Dataset({
    "tbs_tropics":(("samples","angles","channels"),x_valid),
    "tbs_atms":(("samples","angles","channels"),y_valid)
})

ds.to_netcdf('last_test.h5',)

print('Did it work?')